# Spelling Orthography Update

This project uses machine learning to update text from an older spelling orthography to a modern spelling orthography. The focus on this project will be on updating the spelling orthography from the 1908 Chamorro Bible.

**Name:** Schyuler Lujan<br>
**Date Started:** 6-Nov-2024<br>
**Date Completed:** In Progress

In [ ]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Scrape Text Data

Scrape the text data from the chamorrobible.org website and format the text into a dataset of sentences, a dataset of verses, and a dataset of individual words (both duplicated and unduplicated) for different types of analysis. We will also be tagging each sentence, verse and word with its associated book name and book chapter for further text analysis.

In [ ]:
# Assess the structure of the website

# Clean Text Data

In [ ]:
# Remove numbers

In [ ]:
# Remove punctuation

In [ ]:
# Convert to lowercase

# Exploratory Analysis

## Basic Descriptive Statistics

In [ ]:
# Word counts - total and unique

In [ ]:
# Sentence counts

In [ ]:
# Character counts

## Frequency analysis

In [ ]:
# Frequency of words - across entire text and by book

## N-Grams Analysis

In [ ]:
# Commonly occuring word pairs or phrases

# Export Data to Create Training Set

A small, manually created dataset of sample pairs will be created from the text. It will contain the old orthography spelling mapped to the equivalent new orthography spelling, and this dataset will be used to train our machine learning models.

# Train Models

# Evaluale Model Performance

# Final Model Selection

# Export Final Dataset

# Conclusions